In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, Model

# --- Load & preprocess ---
X, y = load_breast_cancer(return_X_y=True)
y = (y == 0).astype(int)  # malignant=1 (anomaly)
X = StandardScaler().fit_transform(X).astype(np.float32)
X_train = X[y == 0]  # train only on normals

# --- VAE ---
class VAE(Model):
    def __init__(self, input_dim=30, latent_dim=8):
        super().__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(32, activation="relu"),
            layers.Dense(latent_dim*2)
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(32, activation="relu"),
            layers.Dense(input_dim)
        ])
    def call(self, x):
        h = self.encoder(x)
        mean, log_var = tf.split(h, 2, axis=1)
        eps = tf.random.normal(tf.shape(mean))
        z = mean + tf.exp(0.5*log_var)*eps
        x_recon = self.decoder(z)
        recon = tf.reduce_mean(tf.reduce_sum(tf.square(x-x_recon), axis=1))
        kl = -0.5*tf.reduce_mean(tf.reduce_sum(1+log_var-tf.square(mean)-tf.exp(log_var), axis=1))
        self.add_loss(recon+kl)
        return x_recon

vae = VAE(input_dim=X.shape[1])
vae.compile(optimizer="adam")
vae.fit(X_train, X_train, epochs=50, batch_size=32, verbose=0)

# --- Anomaly detection ---
X_recon = vae.predict(X)
scores = np.mean(np.square(X - X_recon), axis=1)
th = np.percentile(scores[y==0], 95)  # threshold
preds = (scores > th).astype(int)

# --- Evaluation ---
print("ROC AUC:", roc_auc_score(y, scores))
print("Confusion:\n", confusion_matrix(y, preds))


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC: 0.9568997410284868
Confusion:
 [[339  18]
 [ 36 176]]
